In [1]:
#I crawled the craigslist for couple of days to find the used Toyota information
#Below is my analysis to figure out the average price 
# File location and type
file_location = "/FileStore/tables/deals.jl"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.show(5)

+------+--------------------+--------------------+--------+
 Price| Title| Url|odometer|
+------+--------------------+--------------------+--------+
$36988|2016 Toyota Highl...|https://portland....| 31745|
$36988|2016 Toyota Highl...|https://portland....| 31745|
 $9995|2008 Toyota Highl...|https://portland....| clean|
$17000|2010 Toyota Highl...|https://portland....| hybrid|
 $8995|2011 Toyota High...|https://portland....| gas|
+------+--------------------+--------------------+--------+
only showing top 5 rows

In [2]:

table_name = "car_deals"
df.write.mode('OverWrite').format('parquet').saveAsTable(table_name)


In [3]:
from pyspark.sql.functions import avg, col, udf,desc

@udf('integer')
def convert_value(value):
  try:
    return int(value.replace('$',''))
  except ValueError:
    return None
  


In [4]:
df.count()

Out[ 25 ]: 603

In [5]:
df=df.distinct()
df.count()

Out[ 26 ]: 493

In [6]:
one_car_avg_price= df\
  .select(convert_value('price').alias('price'),'Url')\
  .groupBy('Url')\
  .agg(avg('price'))


In [7]:
one_car_avg_price.show(5)

+--------------------+----------+
 Url|avg(price)|
+--------------------+----------+
https://portland....| 14995.0|
https://portland....| 4500.0|
https://portland....| 6800.0|
https://portland....| 5800.0|
https://portland....| 5250.0|
+--------------------+----------+
only showing top 5 rows

In [8]:
display(
  one_car_avg_price.select('avg(price)').agg(avg('avg(price)').cast('integer'))
)

CAST(avg(avg(price)) AS INT)
13163
